In [1]:
import sys
sys.path.append('..')

from src.data_loader import download_stock_data, get_data_info
from src.strategy import calculate_sma, generate_signals

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (14,7)


In [2]:
ticker = "RELIANCE.NS"
df = download_stock_data(ticker, period ="5y", interval ="1d")

if df is not None:
    print(f" Downloading {ticker} stock data...")
    print(f"period: {df.index[0].date()} to {df.index[-1].date()}")
else:
    print("Data download failed")
    



Succesfully downloaded 1237 rows of data

Date range: 2021-02-01 to 2026-01-30
period: 2021-02-01 to 2026-01-30


In [3]:
short_window = 10
long_window = 50

df_sma = calculate_sma(df, short_window, long_window)

df_sma[['Close', f'SMA_{short_window}', f'SMA_{long_window}']].tail(10)

Calculated SMA 10 and SMA 50


,Close,SMA_10,SMA_50
Date,,,
2026-01-16 00:00:00+05:30,1457.900024,1484.730005,1531.116006
2026-01-19 00:00:00+05:30,1413.599976,1468.280005,1529.828005
2026-01-20 00:00:00+05:30,1394.000000,1456.920007,1527.922004
2026-01-21 00:00:00+05:30,1404.599976,1446.960010,1526.146003
2026-01-22 00:00:00+05:30,1402.500000,1440.150012,1523.966003
2026-01-23 00:00:00+05:30,1386.099976,1431.230005,1521.470002
2026-01-27 00:00:00+05:30,1380.500000,1420.960010,1518.702002
2026-01-28 00:00:00+05:30,1396.699951,1415.350000,1516.270000
2026-01-29 00:00:00+05:30,1391.000000,1408.569995,1513.702000


In [4]:
df_signals = generate_signals(df_sma)

df_signals[['Close', f'SMA_{short_window}', f'SMA_{long_window}', 'Signal']].tail(10)

Buy signals:18
Sell signals:18


,Close,SMA_10,SMA_50,Signal
Date,,,,
2026-01-16 00:00:00+05:30,1457.900024,1484.730005,1531.116006,0.0
2026-01-19 00:00:00+05:30,1413.599976,1468.280005,1529.828005,0.0
2026-01-20 00:00:00+05:30,1394.000000,1456.920007,1527.922004,0.0
2026-01-21 00:00:00+05:30,1404.599976,1446.960010,1526.146003,0.0
2026-01-22 00:00:00+05:30,1402.500000,1440.150012,1523.966003,0.0
2026-01-23 00:00:00+05:30,1386.099976,1431.230005,1521.470002,0.0
2026-01-27 00:00:00+05:30,1380.500000,1420.960010,1518.702002,0.0
2026-01-28 00:00:00+05:30,1396.699951,1415.350000,1516.270000,0.0
2026-01-29 00:00:00+05:30,1391.000000,1408.569995,1513.702000,0.0


In [7]:
def backtest_strategy(df_signals, initial_capital = 100000):
    
    df_backtest = df_signals.copy()
    df_backtest['Daily_Return'] = df_backtest['Close'].pct_change()
    df_backtest['Strategy_Return'] =df_backtest['Daily_Return']*df_backtest['Position'].shift(1)
    
    df_backtest['Cummulative_Market_Returns'] = (1 + df_backtest['Daily_Return']).cumprod() 
    df_backtest['Cummulative_Strategy_Returns'] = (1 + df_backtest['Strategy_Return']).cumprod()
    
    df_backtest['Market_Value'] = df_backtest['Cummulative_Market_Returns']*initial_capital
    df_backtest['Strategy_Value'] = df_backtest['Cummulative_Strategy_Returns']*initial_capital
    
    df_backtest.fillna({'Market_Value': initial_capital, 'Strategy_Value': initial_capital}, inplace=True)
    
    return df_backtest

df_backtest = backtest_strategy(df_signals)
print("Backtest Completed")



Backtest Completed


In [ ]:
def calculate_metrics(df_backtest):
    
    final_market_value =  df_backtest['Market_Value'].iloc[-1]
    final_strategy_value = df_backtest['Strategy_Value'].iloc[-1]
    
        